# Google Search Console GSC API Examples


### Overview

If you are interested in your organic traffic then the Google Search Console is hands down 
one of the richest datasources.

Here are example scripts for connecting to this valuable datasource.


> SEE PART 1 for how to connect and download GSC data



### About Me

My name is Alton Alexander. I am a Data Science consultant turned entreprenuer building SaaS tools for SEO.

Find more about my free scripts or ask me any question on twitter: @alton_lex

# GSC API Examples:

In [3]:
# load libraries
import requests
import json
from urllib.parse import urlparse

import httplib2
from apiclient import errors
from apiclient.discovery import build

import datetime

from google.oauth2 import service_account
import google.oauth2.credentials
import google.auth.transport.requests

!pip install pandas
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python3 -m pip install --upgrade pip


## 0) Setup service account

In Google Cloud Console you need to create the following resources:

1. A project

2. Enable Access to the GSC API

3. A service account (make note of the Email for the service account)

4. OPTIONAL Add rolls to the service account so that it can access additional resources (ie. BigQuery)

5. Save the Key as JSON for connecting from python


In Google Search Console:

1. add the service account email as a user to each of the properties you want it to access

## 1) Connect with Service Account JSON key

Every request to the GSC API must have user authentication.

We use the service account JSON Key we get credentials.


In [89]:
# local path to the service account key
# service account's email must be added to one or more properties in GSC

#SERVICE_ACCOUNT_FILE = "../gcp-keys/website-analytics-161019-16456165cddc.json"
SERVICE_ACCOUNT_FILE = "./service-account-key.json"

In [109]:
SCOPES = [
    'https://www.googleapis.com/auth/webmasters',
    'https://www.googleapis.com/auth/bigquery'
]
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [110]:
service = build(
    'webmasters',
    'v3',
    credentials=credentials
)

In [92]:
# test by pulling a list of sites
# reminder to add the service account email to every GSC property as a user


response = service.sites().list().execute()
response

{'siteEntry': [{'siteUrl': 'sc-domain:bikelanes.com',
   'permissionLevel': 'siteFullUser'}]}

## 1) Pull a list of pages

This pulls a list of all the pages that are showing on Google Search in the last 30 days:

In [55]:
# define the domain that we are using

# hardcode the domain for the property
#site = "frontanalytics.com"

# or select the first property from the response above
site = response['siteEntry'][0]['siteUrl'].split(":")[1]
site

'bikelanes.com'

In [56]:
# set date for the last 30 days
today = datetime.datetime.today()
startDate = today - datetime.timedelta(days=32)
endDate = today - datetime.timedelta(days=2)

In [57]:
# by page

# recent
# Set the dates in this format "2022-12-01"
data = {
  "startDate": startDate.strftime("%Y-%m-%d"),
  "endDate": endDate.strftime("%Y-%m-%d"),
  "dimensions": "page"
}
res = requests.post("https://www.googleapis.com/webmasters/v3/sites/"+"sc-domain:"+site+"/searchAnalytics/query?access_token="+credentials.token, json=data)

In [59]:
j

[{'keys': ['des moines bike lanes', 'MOBILE', 'kor'],
  'clicks': 0,
  'impressions': 1,
  'ctr': 0,
  'position': 36,
  'url': 'des moines bike lanes'},
 {'keys': ['des moines bike lanes', 'MOBILE', 'pse'],
  'clicks': 0,
  'impressions': 1,
  'ctr': 0,
  'position': 37,
  'url': 'des moines bike lanes'}]

In [60]:

if str(res.status_code) != "200":
    print("error with the api response. verify credentials are active.")

else:
    
    # continue
    j = json.loads(res.text).get('rows',[])
    
    for i in range(len(j)):
        j[i]['url'] = j[i]['keys'][0]

    df_pages = pd.DataFrame(j)

    df_pages = df_pages.sort_values('impressions',ascending=False)

df_pages

,keys,clicks,impressions,ctr,position,url
16,[https://bikelanes.com/des-moines-county/],0,22,0,38.681818,https://bikelanes.com/des-moines-county/
6,[https://bikelanes.com/],0,13,0,83.153846,https://bikelanes.com/
51,[https://bikelanes.com/mashpee/],0,11,0,68.454545,https://bikelanes.com/mashpee/
48,[https://bikelanes.com/los-angeles-county/],0,9,0,80.111111,https://bikelanes.com/los-angeles-county/
76,[https://bikelanes.com/tulsa-county/],0,8,0,60.625000,https://bikelanes.com/tulsa-county/
...,...,...,...,...,...,...
31,[https://bikelanes.com/ingram-wi/],0,1,0,49.000000,https://bikelanes.com/ingram-wi/
30,[https://bikelanes.com/hutsonville/],0,1,0,9.000000,https://bikelanes.com/hutsonville/
29,[https://bikelanes.com/horry-county/],0,1,0,19.000000,https://bikelanes.com/horry-county/
28,[https://bikelanes.com/hordville/],0,1,0,5.000000,https://bikelanes.com/hordville/


## 2) Get all KWs for each page

This pulls every query used for each page:

In [93]:
# set date for the previous 1 days (day before yesterday)
updated_at = datetime.datetime.today()
today = updated_at.date()
startDate = today - datetime.timedelta(days=4)
endDate = today - datetime.timedelta(days=3)

# choose the dimensions you want in your dataset

#dim_list = ["query","device","country"]
dim_list = ["query","device","country"] # Add device and country if desired

# for testing
max_n_pages = 3

df_all_queries = pd.DataFrame()

# Get all the queries
for index, row in df_pages[0:max_n_pages].iterrows():
    
    # get the url of this page
    page_url = row['url']

    # recent
    data = {
      "startDate": startDate.strftime("%Y-%m-%d"),
      "endDate": endDate.strftime("%Y-%m-%d"),
      "dimensions": dim_list,
      "dimensionFilterGroups": [
        {
          "groupType": "and",
          "filters": [
            {
              "dimension": "page",
              "operator": "contains",
              "expression": page_url
            }
          ]
        }
      ]
    }
    res = requests.post("https://www.googleapis.com/webmasters/v3/sites/"+"sc-domain:"+site+"/searchAnalytics/query?access_token="+credentials.token, json=data)

    if str(res.status_code) != "200":
        print("error with the api response. verify credentials are active.")
        continue
        
    # convert the response to a data frame
    j = json.loads(res.text).get('rows',[])

    if(len(j)):
        df_queries = pd.DataFrame(j)
        df_queries['url'] = page_url
        df_queries['property'] = site
        df_queries['start_date'] = startDate
        df_queries['updated_at'] = updated_at

        # By default the keys/dimensions are in a single column, let's split them out into separate columns.
        new_cols = df_queries['keys'].astype(str).str.replace("[","").str.replace("]","")
        new_cols = new_cols.str.split(pat=',',expand=True,n=2)

        # Give the columsn sensible names
        new_cols.columns = dim_list

        # Bring back a key from the intial dataframe so we can join
        new_cols['key'] = df_queries['keys']

        # Get rid of quotation marks or set to None
        new_cols['query'] = new_cols.apply(lambda row : row['query'].replace("'","").lower(), axis=1)
        
        if 'device' in new_cols:
            new_cols['device'] = new_cols.apply(lambda row : row['device'].replace("'","").lower(), axis=1)
        else:
            new_cols['device'] = None
        if 'country' in new_cols:
            new_cols['country'] = new_cols.apply(lambda row : row['country'].replace("'","").lower(), axis=1)
        else:
            new_cols['country'] = None

        # Join in the new clean columns to our intiial data
        df_queries = pd.concat([df_queries, new_cols], axis=1, join='inner')
        
        # Drop the key column and ctr
        df_queries = df_queries.drop(["key","keys", "ctr"],axis=1)

        # save all the queries for this page with all other pages
        df_all_queries = pd.concat([df_all_queries, df_queries])

/tmp/ipykernel_324766/2371865715.py:58: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  new_cols = df_queries['keys'].astype(str).str.replace("[","").str.replace("]","")
/tmp/ipykernel_324766/2371865715.py:58: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  new_cols = df_queries['keys'].astype(str).str.replace("[","").str.replace("]","")


In [96]:
# Now you can save df_all_queries for additional analysis
df_all_queries

,clicks,impressions,position,url,property,start_date,updated_at,query,device,country
0,0,1,36,https://bikelanes.com/des-moines-county/,bikelanes.com,2023-01-26,2023-01-30 18:41:46.861954,des moines bike lanes,mobile,kor
1,0,1,37,https://bikelanes.com/des-moines-county/,bikelanes.com,2023-01-26,2023-01-30 18:41:46.861954,des moines bike lanes,mobile,pse
0,0,1,36,https://bikelanes.com/,bikelanes.com,2023-01-26,2023-01-30 18:41:46.861954,des moines bike lanes,mobile,kor
1,0,1,37,https://bikelanes.com/,bikelanes.com,2023-01-26,2023-01-30 18:41:46.861954,des moines bike lanes,mobile,pse
2,0,1,85,https://bikelanes.com/,bikelanes.com,2023-01-26,2023-01-30 18:41:46.861954,miami bike lanes,desktop,uzb


# Configure BigQuery

In [97]:
!pip install --upgrade google-cloud
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade google-cloud-storage
from google.cloud import bigquery

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [98]:

# establish a BigQuery client
client_bq = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_FILE)

In [99]:
# connect to big query and list all datasets

datasets = client_bq.list_datasets()

for dataset in datasets:
    print(dataset.dataset_id)

data_winners_dataset


In [100]:
# get the project name from the service account

BQ_PROJECT_NAME = client_bq.project
BQ_PROJECT_NAME

'bike-lanes-gsc-api-to-bq'

## Create the BQ Dataset and Table

In [102]:
# create the dataset and table if they don't exist yet

BQ_DATASET_NAME = 'data_winners_dataset'
BQ_TABLE_NAME = 'gsc_daily_table'


# create the dataset
try:
    # Construct a full Dataset object to send to the API.
    dataset = bigquery.Dataset("{}.{}".format(client_bq.project, BQ_DATASET_NAME))

    # TODO(developer): Specify the geographic location where the dataset should reside.
    dataset.location = "US"

    # Send the dataset to the API for creation, with an explicit timeout.
    # Raises google.api_core.exceptions.Conflict if the Dataset already
    # exists within the project.
    dataset = client_bq.create_dataset(dataset, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(client_bq.project, dataset.dataset_id))
except Exception as e:
    print(e)

# create the table
# https://cloud.google.com/bigquery/docs/samples/bigquery-create-table 
try:
    # TODO(dev): Change table_id to the full name of the table you want to create.
    #table_id = "your-project.your_dataset.your_table_name"
    table_id = "{}.{}.{}".format(client_bq.project, BQ_DATASET_NAME, BQ_TABLE_NAME)
    
    #SCHEMA must match datatypes to be loaded
    # see. https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
    schema = [
        bigquery.SchemaField("start_date", "DATE", mode="REQUIRED"),
        bigquery.SchemaField("url", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("impressions", "INTEGER", mode="REQUIRED"),
        bigquery.SchemaField("position", "FLOAT64", mode="REQUIRED"),
        bigquery.SchemaField("query", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("device", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("country", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("clicks", "INTEGER", mode="NULLABLE"),
        bigquery.SchemaField("updated_at", "DATETIME", mode="REQUIRED"),
        bigquery.SchemaField("property", "STRING", mode="REQUIRED"),
    ]

    table_description = "Stores GSC data every day. Auto generated via data-winners. For details see https://github.com/FrontAnalyticsInc/data-winners/tree/main/datasource-api-google-search-console"
    
    table = bigquery.Table(table_id, schema=schema)
    table.description = table_description
    table = client_bq.create_table(table)  # API request

    print(f"Created {table_id}.")
except Exception as e:
    print(e)

409 POST https://bigquery.googleapis.com/bigquery/v2/projects/bike-lanes-gsc-api-to-bq/datasets?prettyPrint=false: Already Exists: Dataset bike-lanes-gsc-api-to-bq:data_winners_dataset
409 POST https://bigquery.googleapis.com/bigquery/v2/projects/bike-lanes-gsc-api-to-bq/datasets/data_winners_dataset/tables?prettyPrint=false: Already Exists: Table bike-lanes-gsc-api-to-bq:data_winners_dataset.gsc_daily_table


# Test Uploading to BQ

Make sure your service account has a role that permits access to BigQuery

In [103]:

# Set the destination table
table_id = '{}.{}.{}'.format(BQ_PROJECT_NAME, BQ_DATASET_NAME, BQ_TABLE_NAME)


# create a load job config
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        #bigquery.SchemaField("start_date", bigquery.enums.SqlTypeNames.DATE),
        #bigquery.SchemaField("updated_at", bigquery.enums.SqlTypeNames.DATETIME),
        # Indexes are written if included in the schema by name.
        #bigquery.SchemaField("wikidata_id", bigquery.enums.SqlTypeNames.STRING),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_APPEND",
)

job = client_bq.load_table_from_dataframe(
    df_all_queries, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.



LoadJob<project=bike-lanes-gsc-api-to-bq, location=US, id=1322acbb-2499-49b4-9c91-e5a3a6d14a3a>

# Preparing for Automation

In [107]:
!pip install pandas-gbq
!pip install db-dtypes
import dateutil
import pandas_gbq

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [112]:
# Query bigquery to determine what has already been loaded?
# Update the in-memory credentials cache (added in pandas-gbq 0.7.0).
pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = BQ_PROJECT_NAME


def query_table(start_date, PROJECT_ID, DATASET, TABLE):
    QUERY = "SELECT * FROM {dataset}.{table} WHERE start_date = '{d}'"
    
    query = QUERY.format(dataset=DATASET, table=TABLE, d=start_date)
    result = pd.read_gbq(query, PROJECT_ID, dialect='standard')

    result['already_loaded'] = True
    return result


# get all the records already loaded within the last n days
already_loaded = query_table(startDate, BQ_PROJECT_NAME, BQ_DATASET_NAME, BQ_TABLE_NAME)

already_loaded

,start_date,url,impressions,position,query,device,country,clicks,updated_at,property,already_loaded
0,2023-01-26,https://bikelanes.com/des-moines-county/,1,36.0,des moines bike lanes,mobile,kor,0,2023-01-30 23:58:50.923749,bikelanes.com,True
1,2023-01-26,https://bikelanes.com/,1,36.0,des moines bike lanes,mobile,kor,0,2023-01-30 23:58:50.923749,bikelanes.com,True
2,2023-01-26,https://bikelanes.com/des-moines-county/,1,37.0,des moines bike lanes,mobile,pse,0,2023-01-30 23:58:50.923749,bikelanes.com,True
3,2023-01-26,https://bikelanes.com/,1,37.0,des moines bike lanes,mobile,pse,0,2023-01-30 23:58:50.923749,bikelanes.com,True
4,2023-01-26,https://bikelanes.com/,1,85.0,miami bike lanes,desktop,uzb,0,2023-01-30 23:58:50.923749,bikelanes.com,True
5,2023-01-26,https://bikelanes.com/miami/,1,85.0,miami bike lanes,desktop,uzb,0,2023-01-30 23:58:50.923749,bikelanes.com,True
6,2023-01-26,https://bikelanes.com/des-moines-county/,1,36.0,des moines bike lanes,mobile,kor,0,2023-01-30 18:41:46.861954,bikelanes.com,True
7,2023-01-26,https://bikelanes.com/,1,36.0,des moines bike lanes,mobile,kor,0,2023-01-30 18:41:46.861954,bikelanes.com,True
8,2023-01-26,https://bikelanes.com/des-moines-county/,1,37.0,des moines bike lanes,mobile,pse,0,2023-01-30 18:41:46.861954,bikelanes.com,True
9,2023-01-26,https://bikelanes.com/,1,37.0,des moines bike lanes,mobile,pse,0,2023-01-30 18:41:46.861954,bikelanes.com,True


In [113]:
# find the rows that aren't loaded yet
join_cols = ['start_date','property','url','device','country']
not_loaded = df_all_queries.merge(already_loaded[join_cols+['already_loaded']], how="left", on=join_cols)
not_loaded = not_loaded[not_loaded['already_loaded'].isna()]
cols = list(df_all_queries.columns)
not_loaded = not_loaded[cols]
not_loaded

,clicks,impressions,position,url,property,start_date,updated_at,query,device,country
